In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import csv
import sys

import pandas as pd
from scipy import integrate
from scipy.interpolate import interp1d
from scipy.interpolate import CubicSpline

import dask
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
import dask.dataframe as dd
import time
import psutil

sys.setrecursionlimit(50000)

import os
print(os.getcwd())

/blue/hennig/pawanprakash/anisotropic_eliash/parallaize_lambda


In [ ]:
@dask.delayed
def compute_element(n, m, ωn, alpha2F, omega_val, bin_num):
    """Compute an element of the matrix based on the provided parameters."""
    def integrand(ω):
        return (2 * ω * alpha2F(ω)) / ((ωn[n] - ωn[m]) ** 2 + ω ** 2)

    result, _ = integrate.quad(integrand, omega_val[0], omega_val[-1], limit=bin_num)
    return result

@dask.delayed
def get_unique_elements(ωn, alpha2F, omega_val, bin_num):
    """Compute unique matrix elements based on the distance from the diagonal."""
    N = len(ωn)
    unique_elements = {}
    for dist in range(N):
        unique_elements[dist] = compute_element(0, dist, ωn, alpha2F, omega_val, bin_num)
    return unique_elements

# this fn create the matrix of lambda, but first compute the unique elements(with daskdelay)
#@dask.delayed
def create_matrix(unique_element):
    unique_elements = dask.compute(unique_element)[0]
    N = len(unique_elements)
    """Assign matrix elements based on the unique values and distance from the diagonal."""
    λ_matrix_delayed = np.empty((N, N), dtype=object)
    for i in range(N):
        for j in range(i, N):
            dist = j - i
            λ_matrix_delayed[i, j] = λ_matrix_delayed[j, i] = unique_elements[dist]
    return λ_matrix_delayed


In [ ]:
# define computing cluster and scale jobs, look at development node resources
job_extra = ['--ntasks=8', '--nodes=1', '--qos=hennig-b']
cluster = SLURMCluster(cores=1,
                           memory='64GB',
                           account='hennig',
                           queue='hpg-default',
                           walltime='15:00:00',
                           job_extra_directives= job_extra)

client = Client(cluster)

number_of_workers = 8
cluster.scale(number_of_workers)

In [ ]:
client.wait_for_workers(number_of_workers)

In [ ]:

# coulomb pseudopotential
mu = 0.16
print("mu=", mu)

# load calculated alpha2F
# Column names
columns = ["e(THz)", "a2F(tot)", "a2F(atm1)", "a2F(atm2)", "..."]
# Read the data and set the column names
df = pd.read_csv('./mgb2/040.a2F.dat', delim_whitespace=True, header=None, names=columns, comment='#')
# change the units, THz = 0.0041 eV
df["e(eV)"] = df["e(THz)"] * 0.0041
last_true_index = df['a2F(tot)'].ne(0)[::-1].idxmax()
print("last_true_index=", last_true_index)
# Truncate the DataFrame up to the last non-zero index
df = df.loc[:last_true_index+5, ['e(eV)', 'a2F(tot)']]
# first column is omega_val
omega_val = df.iloc[:, 0].values
# second column is alpha2F_values
a2f_val = df.iloc[:, 1].values
bin_num = (len(omega_val) - 1)

# no of matrubara freq as a multiple of phonon frequency
max_ph_freq_index = df.iloc[:, 1].idxmax()
print("max_ph_freq_index= ", max_ph_freq_index)
max_ph_freq = df.iloc[:, 1][max_ph_freq_index]
print("max_ph_freq= ", max_ph_freq)
no_matsu_ph = (max_ph_freq/(np.pi*0.001) - 1)*0.5 # 0.001 is temperature, chosen for convenience
no_matsu_ph = int(no_matsu_ph) + 1
n = 15*no_matsu_ph # 15 is # here, ie (2*#) * max ph freq is total number of matsubara frequency
print("n=", n)
m = 2 * n # this is total number of matsubara freq, (2*#) times the max ph
print("Total no of Matsubara freq=", m)

# components of anisotropy
x = np.linspace(0, np.pi/2, m)
print("size of ak=", x.shape)
akp = np.sin(x)
print("akp size=", akp.size)
ak = np.sin(x)
print("ak size=", ak.size)

# initialization
#Temp = np.linspace(0.000431, 0.00431, 100)
T = 0.0008

In [ ]:
omega_val_shared = client.scatter(omega_val, broadcast=True)
a2f_val_shared = client.scatter(a2f_val, broadcast=True)
omega = np.asarray([(2 * (-n + i) + 1) * np.pi * T for i in range(m)])
print("omega_shape=", omega.shape)

In [ ]:
alpha2F_future = client.submit(compute_spline, omega_val_shared, a2f_val_shared)
alpha2F_spline = alpha2F_future.result()

In [ ]:
unique_elements_future = get_unique_elements(omega, alpha2F_spline, omega_val_shared, bin_num)

In [ ]:
λ_matrix = create_matrix(unique_elements_future)

In [ ]:
λ_matrix.shape